In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# This file contains all the main external libs we'll use
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

PATH = "data/whales/"
arch = resnext101_64

In [2]:
labels_csv = f'{PATH}train.csv'
amended_csv = f'{PATH}fixed.csv'

In [12]:
labels = pd.read_csv(labels_csv)
labels.head()

,Image,whaleID
0,w_7812.jpg,whale_48813
1,w_4598.jpg,whale_09913
2,w_3828.jpg,whale_45062
3,w_8734.jpg,whale_74162
4,w_3251.jpg,whale_99558


Need to remove an entry for a missing image

In [27]:
missing_idx = labels[labels['Image']=='w_7489.jpg'].index.item()
update_labels = labels.drop(labels.index[missing_idx])
update_labels.to_csv(amended_csv, index=False)
# Now reload labels
labels = pd.read_csv(amended_csv)

In [5]:
# How many of each whale
labels.pivot_table(index='whaleID', aggfunc=len).sort_values('Image', ascending=False)

,Image
whaleID,
whale_95370,47
whale_38681,43
whale_28892,33
whale_90957,30
whale_36851,30
whale_24458,29
whale_85464,28
whale_51195,28
whale_65586,27


In [9]:
n = len(labels) - 1
val_idxs = get_cv_idxs(n)

In [14]:
def get_data(sz, bs):
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_top_down, max_zoom=1.1)
    data = ImageClassifierData.from_csv(PATH, 'imgs', amended_csv,
                                        val_idxs=val_idxs, tfms=tfms, bs=bs)
    return data if sz>300 else data.resize(340, 'tmp')

In [15]:
# Start with a small image to see how things go.
data = get_data(128, 64)

 76%|███████▌  | 2760/3636 [01:20<00:25, 34.33it/s]

FileNotFoundError: [Errno 2] No such file or directory: 'data/whales/imgs/w_7489.jpg'

 76%|███████▌  | 2760/3636 [01:38<00:31, 27.96it/s]

In [ ]:
tf = PATH + data.trn_ds.fnames[0]; fn